In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import subprocess

# Chemin du répertoire de téléchargement

download_dir = "/content/cri_pdf"

# Créer un répertoire s'il n'existe pas déjà
if not os.path.exists(download_dir):
    os.makedirs(download_dir)

def download_with_curl(url, download_dir, filename):
    file_path = os.path.join(download_dir, filename)
    subprocess.run(["curl", "-o", file_path, url], check=True)

driver = webdriver.Chrome()

driver.get('https://www.senat.fr/seances/seances.html')

time.sleep(5)  # Attendre 5 secondes pour que le website se charge complètement

# Créer un objet ActionChains pour faire défiler la page vers le bas
actions = ActionChains(driver)

# XPath du bouton que vous souhaitez cliquer

xpath_to_button = '/html/body/main/article/div/div/div/div[1]/div/div[2]/div[1]/div/div/div[3]/h2/button'

while True:
    try:
        # Utiliser WebDriverWait pour attendre que le bouton soit disponible et cliquer dessus
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, xpath_to_button))
        ).click()

        time.sleep(3)

        # Trouver l'élément qui contient tous les liens
        xpath_to_links_container = '/html/body/main/article/div/div/div/div[1]/div/div[2]/div[1]/div/div/div[3]'

        # Sử dụng WebDriverWait để chờ phần tử chứa liên kết có sẵn
        links_container = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath_to_links_container))
        )
        # Tìm tất cả các link trong phần tử chứa liên kết
        links = links_container.find_elements(By.TAG_NAME, 'a')

        # Lưu tab hiện tại - để mở tab khác -> còn quay lại
        original_tab = driver.current_window_handle

        # Nhấp vào từng liên kết
        for link in links:
            try:
                href = link.get_attribute('href')  # Lấy URL của liên kết
                if href:  # Kiểm tra nếu liên kết không phải là None
                    driver.get(href)  # Mở liên kết
                    time.sleep(2)

                    # Tìm tất cả các liên kết "et au format PDF"
                    pdf_links = driver.find_elements(By.PARTIAL_LINK_TEXT, "et au format PDF")
                    for pdf_link in pdf_links:
                        try:
                            pdf_href = pdf_link.get_attribute('href')
                            if pdf_href:
                                # In URL của tệp PDF
                                print(f"URL của tệp PDF: {pdf_href}")

                                # Obtenir le nom du fichier
                                file_base = pdf_href.split('/')[-1].split('.')[0]  # Partie devant point.
                                new_filename = f"{file_base}.pdf"  # Nom de nouveaux fichier

                                # Télécharger PDF avec nom nouveau fichier
                                download_with_curl(pdf_href, download_dir, new_filename)

                                time.sleep(5)

                        except Exception as e:
                            print(f"Không thể tải tệp PDF: {e}")

                    driver.back()  # Retour de page avant
                    time.sleep(2)
            except Exception as e:
                print(f"Không thể nhấp vào liên kết: {e}")

        break
    except Exception as e:
        # S'il n'a pas trouvé l'élément, faites défiler la page vers le ba
        actions.send_keys(Keys.PAGE_DOWN).perform()
        time.sleep(1)

driver.quit()